# Users With Late Items

We'd like to reach out to these folks and try and get the items back in circulation.

In [ ]:
# What do I want to include?
#
# The member's first and last name
# The member's email and phone number
# The name of item's they have checked out
# When they originally checked them out
# Whether we contacted them before
# How many of that kind of item we have in stock?
import pandas as pd
from random import random
from datetime import datetime, timedelta

def nicer_string(series):
    return ", ".join(set(series.map(str)))

# See README for instructions on downloading fresh data
users = pd.read_csv('data/input_with_personal_info/users-snapshot.csv', usecols=('Member ID', 'Username', 'First Name', 'Last Name', 'Email', 'Confirmed?', 'Phone', 'User Note', 'User Warning'))
users.rename(columns={"Member ID": "Membership ID", 'Confirmed?': 'Email Confirmed?'}, inplace=True)
loans = pd.read_pickle('data/output/loans.pkl')
# TODO: Remove offset
checked_out = loans[loans['Checked In'].isna() & (loans['Due Date'] < (datetime.now() - timedelta(days=1)))]
items = pd.read_pickle('data/output/inventory.pkl')[['Item ID', 'Item Type', 'Name', 'Image','Disabled', 'In Maintenance', 'Shop Use Only', 'Wish List', 'Lost In Shop', 'Lost By Member', 'Not Fixable']]
item_costs = pd.read_csv('data/input/item-cost-guesses.csv', usecols=('id', 'estimated_cost'))
item_costs.rename(columns={'id': 'Item ID', 'estimated_cost': 'Estimated Cost'}, inplace=True)

checked_out_with_details = pd.merge(checked_out, items, how='left', on=['Item ID'])
checked_out_with_details = pd.merge(checked_out_with_details, item_costs, how='left', on=['Item ID'])
checked_out_with_details = checked_out_with_details[~(checked_out_with_details['Disabled'] | checked_out_with_details['Lost In Shop'] | checked_out_with_details['Lost By Member']  | checked_out_with_details['Not Fixable'])]
checkouts_by_member = checked_out_with_details.groupby('Membership ID').agg({'Due Date': "min", 'Name': nicer_string, 'Item ID': nicer_string, 'Estimated Cost': 'sum'})
all_in = pd.merge(users, checkouts_by_member, how="inner", on=["Membership ID"])
all_in.rename(columns={'Name': 'Item Names', 'Item ID': 'Item IDs', 'Estimated Cost': 'AI Guess At $ Value 🤖'}, inplace=True)
all_in['High Value?'] = all_in['AI Guess At $ Value 🤖'].map(lambda v: v >= 200)
all_in['Sort By This To Shuffle'] = all_in['Item IDs'].map(lambda v: random())
# TODO: Add handling for Previously Contacted (match common "contact" words like email, phone, etc on user note and user warning
# TODO: Add handling for "short on item". This is currently just a list, but it would be nice to say something like "demand for this kind of thing
# is higher than the number of items we have in stock

# Drop the Member ID column for the output, since meant to be human readable. Just use username.
all_in.drop(['Membership ID'], axis=1, inplace=True)
all_in.to_csv('data/output_with_personal_info/overdue_loans.csv', index=False)